# GRT Parallel Processing

In this notebook we will test the parallel processing for GRT

In [63]:
from gravray import *
from gravray.util import *
from gravray.sampling import *
from gravray.spice import *
from gravray.orbit import *
from gravray.stats import *

from tqdm import tqdm
import pandas as pd
import multiprocessing as mp
from itertools import product as cartesian

def chunks(l, n):
    for i in range(0, len(l), n):yield l[i:i+n]

Spice.loadKernels()
NP=mp.cpu_count()
print("Number of processors: ",NP)

Number of processors:  4


In [64]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
#import ray
#np=psutil.cpu_count(logical=False)
#ray.init(num_cpus=2)

## Common data

In [66]:
body="EARTH"
earth=Body(body)

## Parallel function

In [71]:
def rayProcessing(initial):
    t=initial[0]
    site=initial[1]
    direction=initial[2]
    ray=GrtRay(site,direction[0][0],direction[0][1],direction[1])
    ray.updateRay(t)
    try:
        ray.propagateRay(t)
        detJ=ray.calcJacobianDeterminant()
    except AssertionError as e:
        detJ=0
    raydf=ray.packRay()
    raydf["detJ"]=detJ
    return raydf
    
def rayProcessingMulti(initials):
    print(f"Processing {len(initials)} initial conditions")
    raydfs=[rayProcessing(initial) for initial in initials]
    return raydfs

## Test data

In [72]:
ts=[Spice.str2t("02/15/2013 03:20:34 UTC")]
siteprops=[[61.1*Angle.Deg,54.8*Angle.Deg,23.3*Const.km]]
sites=[]
for siteprop in siteprops:
    sites+=[Location(earth,siteprop[0],siteprop[1],siteprop[2])]
directions=[[[101.1*Angle.Deg,15.9*Angle.Deg],-18.6*Const.km/Const.s]]

#List of conditions
initials=list(cartesian(*[ts,sites,directions]))
rayProcessingMulti(initials)

Processing 1 initial conditions


[            et   lon   lat      alt      A     h        v          ximp  \
 0  414170434.0  61.1  54.8  23300.0  101.1  15.9 -18600.0 -1.232675e+11   
 
            yimp          zimp      ...           vzhel         q         e  \
 0  8.134799e+10 -3.528140e+06      ...      -2274.5822  0.738582  0.549668   
 
           i           W          w          M         a             n  \
 0  4.041579  326.572556  106.86342  21.323547  1.640082  9.479146e-08   
 
            detJ  
 0 -4.509452e-13  
 
 [1 rows x 28 columns]]

## Massive input data

In [111]:
#Numbers
Ntimes=Nsites=Npoints=Nvels=10

#Times
print("Preparing times...")
tini=Spice.str2t("02/15/2013 03:20:34")
tend=tini+Const.Year
ts=np.linspace(tini,tend,10)

#Sites
print("Preparing sites...")
elTime(0)
H=23.3*Const.km
points=Sample(Nsites)
points.genUnitSphere()
siteprops=np.zeros((Nsites,3))
siteprops[:,:2]=points.pp[:,1:]
siteprops[:,2]=H*np.ones(Nsites)
sites=[]
for siteprop in siteprops:
    sites+=[Location(earth,siteprop[0],siteprop[1],siteprop[2])]
elTime()

#Directions
print("Preparing directions...")
elTime(0)
gpoints=Sample(Npoints)
gpoints.genUnitHemisphere()
speeds=-np.linspace(11.2,72.0,Nvels)*Const.km/Const.s
directions=list(cartesian(*[gpoints.pp[:,1:].tolist(),speeds]))
elTime()

#Initial conditions
print("Preparing initial conditions...")
elTime(0)
initials=list(cartesian(*[ts,sites,directions]))
elTime()

Ninitials=len(initials)
print(f"Number of initial conditions: {Ninitials} = {len(ts)}(ts)*{len(sites)}(sites)*{len(directions)}(dirs)")

Preparing times...
Preparing sites...
Elapsed time since last call: 8.72397 ms
Preparing directions...
Elapsed time since last call: 1.42312 ms
Preparing initial conditions...
Elapsed time since last call: 3.16596 ms
Number of initial conditions: 10000 = 10(ts)*10(sites)*100(dirs)


In [112]:
print(f"Sequential processing of {Ninitials} rays:")
dt,dtu=elTime(0)
tinitials=initials[:10]
rays=rayProcessingMulti(tinitials)
dt,dtu=elTime()
tpray=dt/len(tinitials)
tupray=tUnit(tpray)
totrays=tpray*Ninitials
toturays=tUnit(totrays)
print(f"Total duration: {dtu[0]} {dtu[1]}, Duration per ray: {tupray[0]} {tupray[1]}")
print(f"Estimated total: {toturays[0]} {toturays[1]}")


Sequential processing of 10000 rays:
Processing 10 initial conditions
Elapsed time since last call: 344.362 ms
Total duration: 344.3620204925537 ms, Duration per ray: 34.43620204925537 ms
Estimated total: 5.7393670082092285 min


## Prepare chunks

In [107]:
npchunk=np.int(np.ceil(Ninitials/NP))
cinitials=[initial for initial in chunks(initials,npchunk)]
Nchunks=len(cinitials)
print(f"{Nchunks} chunks containing {npchunk} initial conditions")
tchunk=tpray*npchunk
tchunku=tUnit(tchunk)
print(f"Estimated time per chunk: {tchunku[0]} {tchunku[1]}")

4 chunks containing 20 initial conditions
Estimated time per chunk: 1.2968921661376953 s


## Parallel processing

In [109]:
allraydfs=pd.DataFrame()
def joinResults(raydfs):
    global allraydfs
    allraydfs=pd.concat(((allraydfs,)+tuple(raydfs)))

In [110]:
elTime(0)
for initials in cinitials:
    p=mp.Process(target=rayProcessingMulti,args=(initials,))
    p.start()
p.join()
elTime()

Processing 20 initial conditions
Processing 20 initial conditions
Processing 20 initial conditions
Processing 20 initial conditions
Elapsed time since last call: 1.18376 s


(1.1837592124938965, [1.1837592124938965, 's'])

In [84]:
print("Number of results:",len(allraydfs))

Number of results: 1250


In [85]:
allraydfs.to_csv("rays_parallel.csv")